In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hazetestvideo/hazevideo1.mp4


In [8]:
import cv2
import numpy as np
from os.path import isfile

def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

In [9]:
def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

def dehaze_images(frame_array):
    dehazed_frames = []
    for frame in frame_array:
        dehazed_frame = atmosdehaze(frame)
        dehazed_frames.append(dehazed_frame)

    # Convert the dehazed frames to an array
    dehazed_array = np.array(dehazed_frames)
    return dehazed_array

In [10]:
def dehazed2video(dehazed_array, pathOut, fps=30):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

In [11]:
def main(input_video, output_video):
    frame_array = video2framesarray(input_video)
    dehazed_array = dehaze_images(frame_array)
    dehazed2video(dehazed_array, output_video)

if __name__ == '__main__':
    input_video = '/kaggle/input/hazetestvideo/hazevideo1.mp4'
    output_video = 'output_video.avi'  # Change the format as needed
    main(input_video, output_video)

In [ ]:
from flask import Flask, request, jsonify
import cv2
import numpy as np

app = Flask(__name__)

def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

# Define the API endpoint for dehazing a video
@app.route('/dehaze', methods=['POST'])
def dehaze_video():
    # Check if the request contains a file
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'})

    file = request.files['file']

    # Check if the file is a video
    if file.filename == '':
        return jsonify({'error': 'No file selected'})

    if file and allowed_file(file.filename):
        # Save the video file
        video_path = 'input_video.mp4'
        file.save(video_path)

        # Process the video
        frame_array = video2framesarray(video_path)
        dehazed_array = dehaze_images(frame_array)

        # Save the dehazed video
        output_video_path = 'output_video.avi'
        dehazed2video(dehazed_array, output_video_path)

        return jsonify({'message': 'Video dehazed successfully'})

    return jsonify({'error': 'Invalid file format'})

if __name__ == '__main__':
    app.run()


In [ ]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
import shutil
import cv2
import numpy as np
import os
import asyncio

app = FastAPI()

def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

def dehaze_images(frame_array):
    dehazed_frames = []
    for frame in frame_array:
        dehazed_frame = atmosdehaze(frame)
        dehazed_frames.append(dehazed_frame)

    # Convert the dehazed frames to an array
    dehazed_array = np.array(dehazed_frames)
    return dehazed_array

def dehazed2video(dehazed_array, pathOut, fps=30):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

def dehaze_video(input_path, output_path):
    frame_array = video2framesarray(input_path)
    dehazed_array = dehaze_images(frame_array)
    dehazed2video(dehazed_array, output_path)

@app.post("/dehaze")
async def dehaze_endpoint(file: UploadFile = File(...), fps: int = Form(...)):
    if file.content_type != "video/mp4":
        return {"error": "Only MP4 videos are supported."}

    input_video = "temp_video.mp4"
    output_video = "output_video.mp4"  # You can change the format as needed

    with open(input_video, "wb") as f:
        shutil.copyfileobj(file.file, f)

    await asyncio.to_thread(dehaze_video, input_video, output_video)

    return FileResponse(output_video, media_type="video/mp4")

if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [3]:
import cv2
import numpy as np

def preprocess_image(image, model_input_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (model_input_shape[2], model_input_shape[1]))  # Resize to match the model's input shape
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Define the post-processing function for dehazing
def postprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = (image * 255).astype(np.uint8)
    return image